In [75]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [76]:
url = 'https://habr.com/ru/post/346198/'
page = requests.get(url)

In [77]:
df = pd.DataFrame(columns=['title', 'text', 'year', 'tags', 'votes', 'views', 'bookmarks', 'comments'])

In [79]:
soup = BeautifulSoup(page.text, 'html5lib')

In [80]:
text = soup.find("div",
    class_="post__text post__text-html").text

In [81]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [ ]:
def document(id):
    page = requests.get('https://habr.com/ru/post/' + str(id) + '/')
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    if soup.find("span", class_= "post__title-text"):
        #title
        info.append(soup.find("span", class_= "post__title-text").text)
        #text
        text = soup.find("div", class_="post__text post__text-html").text
        info.append(cleanhtml(text))
        #year
        info.append(soup.find("span", class_= "post__time").text.split()[2])
        #tags
        tagsshit = soup.find_all("a", class_="inline-list__item-link hub-link")
        tags = []
        for i in range(len(tagsshit)):
            tags.append(tagsshit[i].text)
        info.append(tags)
        

In [116]:
def isvotes(tag):
    return tag.clas and tag.has_attr('onclick')

In [121]:
soup.find(isvotes)

<a class="bmenu__conversion" href="https://habr.com/sandbox/start/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'habr_top_panel', 'become_an_author'); }">Как стать автором</a>

In [115]:
tagsshit = soup.find_all("a", class_="inline-list__item-link hub-link")
tags = []
for i in range(len(tagsshit)):
    tags.append(tagsshit[i].text)
tags

['Python',
 'Визуализация данных',
 'Исследования и прогнозы в IT',
 'Веб-аналитика']

In [84]:
soup.find("div", class_="post__text post__text-html")

<div class="post__text post__text-html" data-io-article-url="https://habr.com/ru/post/346198/" id="post-content-body"><p>Когда был доеден новогодний оливье, мне стало нечего делать, и я решил скачать себе на компьютер все статьи с Хабрахабра (и смежных платформ) и поисследовать. </p><br/>
<p>Получилось несколько интересных сюжетов. Первый из них — это развитие формата и тематики статей за 12 лет существования сайта. Например, достаточно показательна динамика некоторых тем. Продолжение — под катом.</p><br/>
<img src="https://habrastorage.org/webt/fa/ht/ez/fahtezsb64hlor2mkatmlp4q7mq.png" width="540"/><a name="habracut"></a><br/>
<h2 id="process-parsinga">Процесс парсинга</h2><br/>
<p>Чтобы понять, как развивался Хабр, нужно было обойти по все его статьи и выделить из них метаинформацию (например, даты). Обход дался легко, потому что ссылки на все статьи имеют вид "habrahabr.ru/post/337722/", причём номера задаются строго по порядку. Зная, что последний пост имеет номер чуть меньше 350 т

In [85]:
soup.find("span", {"class": "post__time"}).text

'8 января 2018 в 15:54'